In [2]:
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import cv2
from os import listdir
from os.path import isfile, join
import pandas as pd
from datetime import datetime
from sklearn import ensemble
from sklearn import svm
from sklearn import linear_model

## Part 1: Data Cleaning

In [3]:
#load the images into a numpy array.
#inspired by https://stackoverflow.com/questions/33369832/read-multiple-images-on-a-folder-in-opencv-python
mypath = "C:\\Users\\David\\Documents\\train\\" #change this path to wherever your image data is
# the below is a list of individual file names
justfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
#prepare my list of images
images = np.empty(len(justfiles), dtype=object)
#fill the list of images with the dog and cat images
for n in range(0, len(justfiles)):
  images[n] = cv2.imread( join(mypath,justfiles[n]) )

In [4]:
#create an array of images to play around with
tempimg = np.empty(500, dtype=object)
for i in range(500):
    tempimg[i] = images[i]

In [5]:
#load the labels, and prepare an array for a bool of labels, dog == True, cat == False
labels = np.loadtxt("..\\data\\train_label.txt", dtype = 'str')
labelsb = np.empty(len(labels), dtype=bool)

In [6]:
#make labels a better string
for i in range(len(labels)):
    if ('d' in labels[i]):
        labels[i] = 'dog'
    else:
        labels[i] = 'cat'

In [7]:
#fill labelsb
for i in range(len(labels)):
    if ('d' in labels[i]):
        labelsb[i] = True
    else:
        labelsb[i] = False

In [8]:
#make all images gray
g_img = np.empty(len(images), dtype = object)
for i in range(len(images)):
    g_img[i] = cv2.cvtColor(images[i], cv2.COLOR_BGR2GRAY)

In [9]:
#resize all images; make them uniform
gu_img = np.empty(len(g_img), dtype = object)
for i in range(len(images)):
    gu_img[i] = cv2.resize(g_img[i], (128,128))

In [10]:
len(gu_img[0:1400])

1400

In [11]:
#train and test sets
trn = gu_img[0:1400]
tst = gu_img[1400:2000]
# tmp train and test labels
ltrn = labelsb[0:1400]
ltst = labelsb[1400:2000]

### ORB Stuff

In [30]:
# Initiate STAR detector for all the images
orbs = np.empty(len(gu_img), dtype = object)
for i in range(len(gu_img)):
    orbs[i] = cv2.ORB_create()

# find the keypoints with ORB
kps = np.empty(len(gu_img), dtype=object)
for i in range(len(orbs)):
    kps[i] = orbs[i].detect(gu_img[i],None)

# compute the descriptors with ORB
descrs = np.empty(len(kps), dtype=object)
for i in range(len(kps)):
    kps[i],descrs[i] = orbs[i].compute(gu_img[i],kps[i])

In [31]:
dict_size = 5
bow = cv2.BOWKMeansTrainer(dict_size)

for i in range(len(descrs)):
    bow.add(np.float32(descrs[i]))

dictionary = bow.cluster()

In [32]:
dictionary.shape

(5, 32)

In [33]:
orb2 = cv2.ORB_create()
bowDiction = cv2.BOWImgDescriptorExtractor(orb2, cv2.BFMatcher(cv2.NORM_HAMMING))
bowDiction.setVocabulary(dictionary)
print(np.shape(dictionary))

(5, 32)


In [35]:
bowfea = np.empty(len(gu_img), dtype=object)
for i in range(len(gu_img)):
    bowfea[i] = bowDiction.compute(gu_img[i], kps[i], descrs[i])

error: C:\projects\opencv-python\opencv\modules\features2d\src\matchers.cpp:744: error: (-215) _queryDescriptors.type() == trainDescType in function cv::BFMatcher::knnMatchImpl


In [36]:
#surf
surf = cv2.xfeatures2d.SURF_create(400)
surf.extended = True

kps = np.empty(len(tmp_trn), dtype=object)
for i in range(len(kps)):
    kps[i],des[i] = surf.detectAndCompute(tmp_trn[i],None)

AttributeError: module 'cv2.cv2' has no attribute 'xfeatures2d'

### Bag of words on orb features

The following section is from my project partner Yang He.

In [37]:
# Deeper Feature Extraction

# Bag of word
# clustering
def bow_cluster(kmeans_obj, descriptor_stack):
    kmeans_ret = kmeans_obj.fit_predict(descriptor_stack)
    return kmeans_ret

# generate vertical stack of descriptors
def bow_vstack(desc_list):
    stack = np.array(desc_list[0])
    for rest in desc_list[1:]:
        stack = np.vstack((stack, rest))
    desc_stack = stack.copy()
    return desc_stack

# generate bag of words frequency matrix (shape: NUM_TRAIN_IMG * N_CLUSTER)
def bow_get_freq_matrix(num_imgs, num_clusters, SIFT_list, kmeans_ret):
    # initialization
    matrix = np.array([np.zeros(num_clusters) for i in range(num_imgs)])
    
    # keep track of index of kmeans_ret
    kmeans_id = 0
    for i in range(num_imgs):
        l = len(SIFT_list[i])
        for j in range(l):
            cluster_id = kmeans_ret[kmeans_id + j]
            matrix[i][cluster_id] += 1
        kmeans_id += l
    
    return matrix

In [ ]:
descriptor_stack = bow_vstack(descrs)

In [41]:
from sklearn.cluster import KMeans
kmeans_obj = KMeans(n_clusters = 200)
kmeans_ret = bow_cluster(kmeans_obj, descriptor_stack)

In [45]:
vocab_matrix = bow_get_freq_matrix(2000, 200, descrs, kmeans_ret)

In [47]:
from sklearn.preprocessing import StandardScaler

scale = StandardScaler().fit(vocab_matrix)
vocab_matrix_std = scale.transform(vocab_matrix)

In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vocab_matrix_std, labelsb, test_size = 0.2, random_state =42)

In [51]:
start_time = datetime.now()
print("Start training using SVM @ " + str(start_time))

clf = svm.SVC()
clf.fit(X_train, y_train)
pred = np.asarray(clf.predict(X_test))
accuracy = (pred == y_test).mean()
print("Accuracy: %.2f%%" % (accuracy * 100))
end_time = datetime.now()
print("End training @ " + str(end_time) + " Time Cost: " + str(end_time - start_time))

Start training using SVM @ 2018-03-03 18:10:32.355518
Accuracy: 70.00%
End training @ 2018-03-03 18:10:33.105365 Time Cost: 0:00:00.749847


### Train other classifiers with BOW ORB features

In [58]:
start_time = datetime.now()
print("Start training using GBC @ " + str(start_time))

clf = ensemble.GradientBoostingClassifier()
clf.fit(X_train, y_train)

end_time = datetime.now()
print("End training @ " + str(end_time) + " Time Cost: " + str(end_time - start_time))

Start training using GBC @ 2018-03-03 19:37:03.526160
End training @ 2018-03-03 19:37:04.539437 Time Cost: 0:00:01.013277


In [59]:
start_time = datetime.now()
print("Start training using GBC @ " + str(start_time))

pred = np.asarray(clf.predict(X_test))
accuracy = (pred == y_test).mean()
print("Accuracy: %.2f%%" % (accuracy * 100))

end_time = datetime.now()
print("End training @ " + str(end_time) + " Time Cost: " + str(end_time - start_time))

Start training using GBC @ 2018-03-03 19:37:08.910028
Accuracy: 68.25%
End training @ 2018-03-03 19:37:08.913029 Time Cost: 0:00:00.003001


In [60]:
start_time = datetime.now()
print("Start training using RF @ " + str(start_time))

clf = ensemble.RandomForestClassifier()
clf.fit(X_train, y_train)

end_time = datetime.now()
print("End training @ " + str(end_time) + " Time Cost: " + str(end_time - start_time))

Start training using RF @ 2018-03-03 19:37:15.823833
End training @ 2018-03-03 19:37:15.891893 Time Cost: 0:00:00.068060


In [61]:
start_time = datetime.now()
print("Start training using RF @ " + str(start_time))

pred = np.asarray(clf.predict(X_test))
accuracy = (pred == y_test).mean()
print("Accuracy: %.2f%%" % (accuracy * 100))

end_time = datetime.now()
print("End training @ " + str(end_time) + " Time Cost: " + str(end_time - start_time))

Start training using RF @ 2018-03-03 19:37:17.934256
Accuracy: 59.00%
End training @ 2018-03-03 19:37:17.938260 Time Cost: 0:00:00.004004


In [62]:
start_time = datetime.now()
print("Start training using ADB @ " + str(start_time))

clf = ensemble.AdaBoostClassifier()
clf.fit(X_train, y_train)

end_time = datetime.now()
print("End training @ " + str(end_time) + " Time Cost: " + str(end_time - start_time))

Start training using ADB @ 2018-03-03 19:38:29.615501
End training @ 2018-03-03 19:38:30.010053 Time Cost: 0:00:00.394552


In [63]:
start_time = datetime.now()
print("Start training using ADB @ " + str(start_time))

pred = np.asarray(clf.predict(X_test))
accuracy = (pred == y_test).mean()
print("Accuracy: %.2f%%" % (accuracy * 100))

end_time = datetime.now()
print("End training @ " + str(end_time) + " Time Cost: " + str(end_time - start_time))

Start training using ADB @ 2018-03-03 19:38:33.285070
Accuracy: 62.75%
End training @ 2018-03-03 19:38:33.297087 Time Cost: 0:00:00.012017


### HOG Stuff

In [12]:
winSize = (128,128)
blockSize = (16,16)
blockStride = (8,8)
cellSize = (8,8)
nbins = 9

hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins)

des = np.empty(len(trn), dtype=object)
for i in range(len(des)):
    des[i] = hog.compute(trn[i])

In [13]:
des_tst = np.empty(len(tst), dtype=object)
for i in range(len(des_tst)):
    des_tst[i] = hog.compute(tst[i])

In [14]:
(des[0].flatten()).shape

(8100,)

### Clean HOG

In [16]:
#flatten the arrays
f_trn = np.empty(len(trn),dtype=object)
for i in range(len(trn)):
    f_trn[i] = trn[i].flatten()

f_tst = np.empty(len(tst),dtype=object)
for i in range(len(tst)):
    f_tst[i] = tst[i].flatten()
  
f_des = np.empty(len(des),dtype=object)
for i in range(len(f_des)):
    f_des[i] = des[i].flatten()

f_des_tst = np.empty(len(tst), dtype=object)
for i in range(len(f_des_tst)):
    f_des_tst[i] = des_tst[i].flatten()

In [17]:
#reformat the arrays
f_trn = np.vstack(f_trn)
f_tst = np.vstack(f_tst)
f_des = np.vstack(f_des)
f_des_tst = np.vstack(f_des_tst)

In [18]:
f_des.shape

(1400, 8100)

### Run Models With HOG

In [19]:
#fit the logistic reggresion modelmodel
lrmodel = linear_model.LogisticRegression(solver='sag')
lrmodel.fit(f_des,ltrn)#lrmodel.fit(ftmp_trn,tmp_ltrn)

C:\Users\David\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False)

In [20]:
1 - (sum(lrmodel.predict(f_des_tst) ^ ltst)/600)

0.59333333333333327

In [22]:
#fit a gradient boosting algorithm
start_time = datetime.now()

gbcmodel = ensemble.GradientBoostingClassifier()
gbcmodel.fit(f_des,ltrn)

end_time = datetime.now()
print("complete @ " + str(end_time) + " Time Cost: " + str(end_time - start_time))

complete @ 2018-03-03 13:26:43.006803 Time Cost: 0:01:55.547045


In [23]:
start_time = datetime.now()

acc = 1 - (sum(gbcmodel.predict(f_des_tst) ^ ltst)/600)

end_time = datetime.now()
print("complete @ " + str(end_time) + " Time Cost: " + str(end_time - start_time))
print(acc)

complete @ 2018-03-03 13:27:05.899684 Time Cost: 0:00:00.016026
0.63


In [28]:
#fit adaboost
start_time = datetime.now()

adbmodel = ensemble.AdaBoostClassifier()
adbmodel.fit(f_des,ltrn)

end_time = datetime.now()
print("complete @ " + str(end_time) + " Time Cost: " + str(end_time - start_time))

complete @ 2018-03-03 13:33:37.055442 Time Cost: 0:01:04.129575


In [29]:
start_time = datetime.now()

acc = 1 - (sum(adbmodel.predict(f_des_tst) ^ ltst)/600)

end_time = datetime.now()
print("complete @ " + str(end_time) + " Time Cost: " + str(end_time - start_time))
print(acc)

complete @ 2018-03-03 13:33:52.011831 Time Cost: 0:00:00.181946
0.625


In [24]:
#fit random forest
start_time = datetime.now()

rf = ensemble.RandomForestClassifier()
rf.fit(f_des,ltrn)

end_time = datetime.now()
print("complete @ " + str(end_time) + " Time Cost: " + str(end_time - start_time))

complete @ 2018-03-03 13:27:16.673594 Time Cost: 0:00:01.025556


In [25]:
start_time = datetime.now()

acc = 1 - (sum(rf.predict(f_des_tst) ^ ltst)/600)

end_time = datetime.now()
print("complete @ " + str(end_time) + " Time Cost: " + str(end_time - start_time))
print(acc)

complete @ 2018-03-03 13:27:19.493271 Time Cost: 0:00:00.009994
0.601666666667


In [26]:
#fit svm
start_time = datetime.now()

svmmodel = svm.SVC(kernel='rbf')
svmmodel.fit(f_des,ltrn)

end_time = datetime.now()
print("complete @ " + str(end_time) + " Time Cost: " + str(end_time - start_time))

complete @ 2018-03-03 13:27:41.761828 Time Cost: 0:00:15.627986


In [27]:
start_time = datetime.now()
acc = 1 - (sum(svmmodel.predict(f_des_tst) ^ ltst)/600)
end_time = datetime.now()
print("complete @ " + str(end_time) + " Time Cost: " + str(end_time - start_time))
print(acc)

complete @ 2018-03-03 13:28:02.375045 Time Cost: 0:00:05.556145
0.676666666667
